In [15]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/Patient-Recovery-Prediction-Challenge/sample_submission.csv
/kaggle/input/Patient-Recovery-Prediction-Challenge/train.csv
/kaggle/input/Patient-Recovery-Prediction-Challenge/test.csv


In [16]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_squared_error


In [17]:
import pandas as pd

# Load the training dataset
train_df = pd.read_csv("/kaggle/input/Patient-Recovery-Prediction-Challenge/train.csv")

# View the first few rows
train_df.head()

,Id,Therapy Hours,Initial Health Score,Lifestyle Activities,Average Sleep Hours,Follow-Up Sessions,Recovery Index
0,9255,5,49,No,7,5,36
1,1562,2,48,Yes,7,6,25
2,1671,2,81,No,7,2,59
3,6088,2,46,No,6,1,22
4,6670,8,47,No,9,0,40


In [18]:
# Drop ID column
train_df = train_df.drop(columns=["Id"])

# Convert "Lifestyle Activities" to numeric binary column
train_df["Lifestyle Activities_Yes"] = train_df["Lifestyle Activities"].apply(lambda x: 1 if x == "Yes" else 0)

# Drop the original column
train_df = train_df.drop(columns=["Lifestyle Activities"])

# Separate features and target
X = train_df.drop(columns=["Recovery Index"])
y = train_df["Recovery Index"]

# Check result
train_df.head()


,Therapy Hours,Initial Health Score,Average Sleep Hours,Follow-Up Sessions,Recovery Index,Lifestyle Activities_Yes
0,5,49,7,5,36,0
1,2,48,7,6,25,1
2,2,81,7,2,59,0
3,2,46,6,1,22,0
4,8,47,9,0,40,0


In [22]:
# All columns are numeric now
numeric_cols = X.columns

# Standardize numeric columnsnumeric_cols = X.columns
preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numeric_cols)
    ]
)

# Define Elastic Net model
elastic_net = ElasticNet(max_iter=10000, random_state=42)

# Create pipeline
model = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("regressor", elastic_net)
])
preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numeric_cols)
    ]
)


In [23]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


In [24]:
param_grid = {
    "regressor__alpha": [0.01, 0.1, 1, 10],
    "regressor__l1_ratio": [0.1, 0.5, 0.7, 0.9]
}

grid_search = GridSearchCV(model, param_grid, cv=5, scoring="neg_mean_squared_error")
grid_search.fit(X_train, y_train)

print("Best Parameters:", grid_search.best_params_)


Best Parameters: {'regressor__alpha': 0.01, 'regressor__l1_ratio': 0.9}


In [25]:
best_model = grid_search.best_estimator_

y_pred = best_model.predict(X_val)

mse = mean_squared_error(y_val, y_pred)
print("Validation MSE:", mse)


Validation MSE: 4.027849067207379


In [27]:
# Load test data
test_df = pd.read_csv("/kaggle/input/Patient-Recovery-Prediction-Challenge/test.csv")

# Convert Lifestyle Activities to binary
test_df["Lifestyle Activities_Yes"] = test_df["Lifestyle Activities"].apply(lambda x: 1 if x == "Yes" else 0)
test_df = test_df.drop(columns=["Lifestyle Activities"])

# Keep Ids for submission
ids = test_df["Id"]
test_df = test_df.drop(columns=["Id"])

# Predict
test_preds = best_model.predict(test_df)

# Save submission
submission = pd.DataFrame({"Id": ids, "Recovery Index": test_preds})
submission.to_csv("Elastic net.csv", index=False)

print("✅ submission.csv created successfully!")


✅ submission.csv created successfully!
